### 合并和拼接
+ merge类似于sql语句里的join

DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes='_x', '_y', copy=True, indicator=False, validate=None)

how:'inner', 'left','right','outer'
+ concat

concat(objs: Union\[Iterable\[FrameOrSeries\], Mapping\[Label, FrameOrSeries\]\], axis='0', join: str = "'outer'", ignore_index: bool = 'False', keys='None', levels='None', names='None', verify_integrity: bool = 'False', sort: bool = 'False', copy: bool = 'True')

+ append

#### 总结
+ merge用于联合查询，将多个表的信息，通过同一个关键字段，关联在一起
+ concat用于增加数据，增加行数据或者列数据

#### DateFrame上的合并
+ 合并对象
+ 合并方式inner，outer等
+ 索引是否重排

In [8]:
import pandas as pd
df1 = pd.DataFrame({'A': [1,2,3], 'B': [4,5,6], 'C': [7,8,9]})
df2 = pd.DataFrame({'A': [11,12], 'B':[12,14], 'D': [15,16]})

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [9]:
df2

,A,B,D
0,11,12,15
1,12,14,16


In [10]:
# 默认使用axis = 0, outter方式
pd.concat([df1,df2])

,A,B,C,D
0,1,4,7.0,NaN
1,2,5,8.0,NaN
2,3,6,9.0,NaN
0,11,12,NaN,15.0
1,12,14,NaN,16.0


In [19]:
# 使用inner join，并且忽略index
pd.concat([df1, df2], join='inner',ignore_index=True)

,A,B
0,1,4
1,2,5
2,3,6
3,11,12
4,12,14


In [21]:
df3 = pd.DataFrame([[21,22,23],[31,32,33]], columns=['E','F','G'])
df3

,E,F,G
0,21,22,23
1,31,32,33


In [25]:
# 在axis=1上进行concat
pd.concat([df1,df3],axis=1)

,A,B,C,E,F,G
0,1,4,7,21.0,22.0,23.0
1,2,5,8,31.0,32.0,33.0
2,3,6,9,NaN,NaN,NaN


In [27]:
s1 = pd.Series(list(range(4)), name='H')
s1

0    0
1    1
2    2
3    3
Name: H, dtype: int64

In [29]:
# 拼接Series
pd.concat([df1,s1],axis=1, join='inner')

,A,B,C,H
0,1,4,7,0
1,2,5,8,1
2,3,6,9,2


In [40]:
# 逐行添加数据
df = pd.DataFrame(columns=list('AB'))
df

,A,B


In [39]:
# 低性能版本
for i in range(4):
    df = df.append({'A': i, 'B': i*2}, ignore_index=True)
df

,A,B
0,0,0
1,1,2
2,2,4
3,3,6


In [46]:
# 高性能版本
pd.concat([pd.DataFrame([i], columns=['A']) for i in range(5)], ignore_index=True)

,A
0,0
1,1
2,2
3,3
4,4


### 实战-分割合并excel
#### 分割excel

In [47]:
import pandas as pd
import os
work_dir = '../course_datas/c15_excel_split_merge'
split_dir = f'{work_dir}/parts'

if not os.path.exists(split_dir):
    os.mkdir(split_dir)

In [49]:
source = pd.read_excel(f'{work_dir}/crazyant_blog_articles_source.xlsx')
source.head()

,id,title,tags
0,2585,Tensorflow怎样接收变长列表特征,"python,tensorflow,特征工程"
1,2583,Pandas实现数据的合并concat,"pandas,python,数据分析"
2,2574,Pandas的Index索引有什么用途？,"pandas,python,数据分析"
3,2564,机器学习常用数据集大全,"python,机器学习"
4,2561,一个数据科学家的修炼路径,数据分析


In [55]:
# source['id'].count()
# source.shape[0]
import math
space = 8
pieces = math.ceil(source.shape[0] / space)
pieces

33

In [59]:
# 分割
prefix = 'user'
for i in range(space):
    name = f'{split_dir}/crazyant_blog_articles_{prefix}_{i}.xlsx'
    sub = source.iloc[i* pieces: i*pieces + pieces]
    sub.to_excel(name, index=False)

In [71]:
# 合并
import re
sub_list = []
rname = r'^crazyant_blog_articles_(\w+\d+)\.xlsx$'
for name in os.listdir(split_dir):
    sub = pd.read_excel(f'{split_dir}/{name}')
    username = re.search(rname, name, re.M).group(1)
    sub['username'] = username
    # print(name, f'{split_dir}/{name}')
    sub_list.append(sub)

In [72]:
df_merged = pd.concat(sub_list)
df_merged.head()

,id,title,tags,username
0,449,[C++]win32输出当前系统时间函数，可用以程序计时,c++,user_6
1,444,[转]office – word2010每次打开弹出配置框解决方法,"office,word",user_6
2,414,Linux下GCC和Makefile实例（从GCC的编译到Makefile的引入）,linux,user_6
3,410,让QT支持中文的方法,qt,user_6
4,408,QT-creater一个非常棒的教程,qt,user_6


In [74]:
df_merged.to_excel(f'{work_dir}/crazyant_blog_articles_concat.xlsx',index=False)

In [79]:
# 每个用户名的条数
df_merged['username'].value_counts()

user_3    33
user_6    33
user_2    33
user_1    33
user_0    33
user_4    33
user_5    33
user_7    27
Name: username, dtype: int64